In [6]:
from kiwipiepy import Kiwi
import re
import time
import json
import time
import html
import pandas as pd
import numpy as np

### 불용어 설정

In [7]:
# 키위 기본 불용어
from kiwipiepy.utils import Stopwords
stopwords = Stopwords()

In [8]:
# 슬랙 질의응답 참고 불용어
stopwords2 = [
    "아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을",
    "를", "에", "의", "가", "으로", "로", "에게", "뿐이다", "의거하여", "근거하여", "입각하여",
    "기준으로", "예하면", "예를", "들면", "예를", "들자면", "저", "소인", "소생", "지말고",
    "하지마", "하지마라", "다른", "물론", "또한", "그리고", "비길수", "없다", "해서는", "안된다",
    "뿐만", "아니라", "만이", "아니다", "만은", "아니다", "막론하고", "관계없이", "그치지", "않다",
    "그러나", "그런데", "하지만", "든간에", "논하지", "않다", "따지지", "않다", "설사", "비록",
    "더라도", "아니면", "만", "못하다", "하는", "편이", "낫다", "불문하고", "향하여", "향해서",
    "향하다", "쪽으로", "틈타", "이용하여", "타다", "오르다", "제외하고", "이", "외에", "이",
    "밖에", "하여야", "비로소", "한다면", "외에도", "이곳", "여기", "부터", "기점으로", "따라서",
    "할", "생각이다", "하려고하다", "이리하여", "그리하여", "그렇게", "함으로써", "일때", "할때",
    "앞에서", "중에서", "보는데서", "으로써", "로써", "까지", "해야한다", "일것이다", "반드시",
    "할줄알다", "할수있다", "할수있어", "임에", "틀림없다", "한다면", "등", "제", "겨우", "단지",
    "다만", "할뿐", "딩동", "댕그", "대해서", "대하여", "대하면", "훨씬", "얼마나", "얼마만큼",
    "얼마큼", "남짓", "여", "얼마간", "약간", "다소", "좀", "조금", "다수", "몇", "얼마", "지만",
    "하물며", "이와", "반대로", "바꾸어서", "말하면", "바꾸어서", "한다면", "몰라도", "외에도", "이렇게",
    "하면", "이렇게", "된다면", "이러한", "즉", "구체적으로", "말하자면", "시작하여", "시초에", "이상",
    "허", "헉", "허걱", "바와같이", "해도좋다", "해도된다", "게다가", "더구나", "하물며", "와르르",
    "팍", "퍽", "펄렁", "동안", "이래", "하고있었다", "이었다", "에서", "로부터", "까지", "예하면",
    "했어요", "해요", "같이", "더불어", "마저", "모두", "습니다", "각", "각각", "여러분", "각종",
    "각자", "제각기", "하도록하다", "와", "과", "그러므로", "그래서", "고로", "한", "까닭에", "하기",
    "때문에", "네", "예", "우선", "누구", "누가", "알겠는가", "아무도", "줄은모른다", "줄은", "몰랏다",
    "하는", "김에", "겸사겸사", "하는바", "그런", "까닭에", "한", "이유는", "그래", "저기", "저쪽",
    "저것", "그때", "그럼", "그러면", "요", "이쪽", "여기", "이것", "이번", "이런", "이러한", "이와",
    "같은", "요", "만큼", "얼마만큼", "얼마큼", "이만큼", "이", "정도의", "이렇게", "많은", "이와",
    "같다", "이때", "것과", "같이", "삐걱", "따위", "와", "같은", "사람들", "부류의", "사람들", "중의",
    "하나", "오직", "오로지", "에", "한하다", "하기만", "하면", "도착하다", "까지", "미치다", "도달하다",
    "정도에", "이르다", "할", "지경이다", "결과에", "이르다", "관해서는", "여러분", "하고", "있다",
    "한", "후", "혼자", "자기", "자기집", "자신", "우에", "종합한것과같이", "총적으로", "보면",
    "총적으로", "말하면", "총적으로", "대로", "하다", "으로서", "참", "그만이다", "할", "따름이다",
    "쿵", "탕탕", "쾅쾅", "둥둥", "봐", "봐라", "아이야", "아니", "와아", "응", "아이", "참나", "년",
    "월", "일", "령", "영", "이", "삼", "사", "오", "육", "륙", "칠", "팔", "구", "이천육", "이천칠",
    "이천팔", "이천구", "하나", "둘", "셋", "넷", "다섯", "여섯", "일곱", "여덟", "아홉", "령", "영",
    "이", "있", "하", "것", "들", "그", "되", "수", "왜", "뭐", "거", "해", "저", "너", "건", "은",
    "걸", "진짜", "보", "않", "없", "사람", "주", "아니", "등", "같", "때", "년", "가", "지", "대하",
    "오", "말", "일", "그렇", "위하", "때문", "그것", "두", "알", "속", "하나", "집", "모르", "적",
    "월", "데", "자신", "점", "싶", "말", "정도", "좀", "원", "잘", "통하", "소리", "놓"
]

### 불용어 제거, 띄어쓰기, 오타교정, 형태소 분석 함수 정의
#### 명사만, 용언만 남기는 함수 정의

In [28]:
kiwi = Kiwi(typos='basic')

In [9]:
def preprocess_hangul(my_str, analyzer=kiwi):
    kiwi.add_user_word('국민의힘', 'NNP')# 국민의힘 사용자 단어 추가
    # 원문 tokenizing(띄어쓰기, 오타교정, 형태소 분석)
    my_str = my_str.replace('\n', ' ') # 줄바꿈 문자 제거
    space_results = kiwi.space(my_str) # 띄어쓰기 교정
    p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]') # 특수 문자 제거 (=한글을 제외한 문자 제거)
    tokens = p.sub(' ', space_results) 
    #token_result = kiwi.join(tokens)
    tokens = kiwi.tokenize(tokens, # 형태소 분석 + 간단한 오타 교정
                           normalize_coda=True, 
                           z_coda=True, 
                           split_complex=False)
    nouns = [token[0] for token in tokens if token[1] in ['NNG', 'NNP']]#명사만
    verbs = [token[0] for token in tokens if token[1] in ['VV', 'VA', 'VX', 'VCP', 'VCN']]#동사만
    results = [token.form for token in tokens if (token.form, token.tag) not in stopwords]
    results = [result for result in results if result not in stopwords2]
    results = [result for result in results if result not in stopwords3] # LDA는 join 과정 제외
    results_WC = ' '.join(results) # 워드클라우드는 모든 문장을 하나의 string으로 join
    return results, results_WC, nouns, verbs # LDA용, 워드클라우드용, 명사, 동사

### 언론사 성향 분류 함수 정의

In [10]:
# 언론사 성향 분류 함수
def classify_agency(agency):
    # 보수 성향 신문사 목록
    conservative_agencies = [
    "조선일보", "동아일보", "중앙일보", "문화일보", "TV조선", "채널A", 
    "한국경제", "매일경제", "헤럴드경제", "이데일리", "서울경제", 
    "머니투데이", "디지털타임스", "아시아경제", "파이낸셜뉴스", 
    "조선비즈", "데일리안", "조세일보", "국민일보", "국회", 
    "미래통합당", "바른정당", "선진통일당",
    "매일신문", "부산일보", "제주일보", "강원일보", "경기일보",
    "국제신문", "대전일보", "매일경제TV", "헤럴드POP", "내일신문",
    "강원도민일보", "시사저널", "신동아", "주간조선"]

    # 진보 성향 신문사 목록
    progressive_agencies = [
    "한겨레", "경향신문", "오마이뉴스", "프레시안", "미디어오늘", "시사IN", 
    "한겨레21", "블로터", "주간경향", "더불어민주당", "정의당",
    "kbc광주방송", "CJB청주방송", "대구MBC", "전주MBC"]

    # 중도 성향 신문사 목록
    moderate_agencies = [
    "연합뉴스", "KBS", "MBC", "SBS", "뉴시스", "YTN", "JTBC", 
    "노컷뉴스", "디지털데일리", "ZDNet Korea", "코리아헤럴드", 
    "코리아중앙데일리", "씨네21", "한국일보", "헤럴드POP", 
    "연합뉴스TV", "EPA연합뉴스", "연합인포맥스", "정책브리핑", 
    "국정브리핑", "해양수산부", "외교통상부", "더스쿠프", "비즈워치",
    "스포츠서울", "스포츠경향", "스포츠동아", "스포츠조선",
    "스포츠월드", "스타뉴스", "TV리포트", "헬스조선", 
    "코리아중앙데일리", "일간스포츠", "SBS CNBC", "청와대",
    "내일신문", "정책브리핑", "헤럴드POP", "쿠키뉴스",
    "코리아헤럴드", "더불어민주당", "선진통일당",
    "EPA연합뉴스", "국회", "미래통합당", "국정브리핑",
    "바른정당", "정의당", "한경비즈니스", "농민신문",
    "비즈워치", "씨네21", "연합인포맥스", "오마이TV",
    "외교통상부", "제주일보", "해양수산부"]

    if agency in progressive_agencies :
        return '진보'
    elif agency in moderate_agencies:
        return '중도'
    elif agency in conservative_agencies:
        return '보수'
    else:
        return '알 수 없음'

### 윤석열 데이터 전처리

In [11]:
# 대통령별 stopword
stopwords3 = ["대통령", "전대통령","주년","정부","무단","전재","배포", "윤석열", "윤 대통령", "윤"]

In [12]:
# 크롤링 데이터 불러오기
yoon = pd.read_csv('news_data_윤_final.csv', encoding = 'utf-8')

In [13]:
# pre열 만들어 LDA용, 워드클라우드용, 명사, 동사 순으로 저장
yoon['pre'] =  yoon['content'].map(lambda x : preprocess_hangul(x))

In [14]:
# LDA용 열
yoon['preprocessed_content_LDA'] =  yoon['pre'].map(lambda x : x[0])

In [15]:
# 워드클라우드용 열
yoon['preprocessed_content_WC'] = yoon['pre'].map(lambda x : x[1])

In [16]:
# 명사열
yoon['content_noun'] = yoon['pre'].map(lambda x : x[2])

In [17]:
# 동사열
yoon['content_verb'] = yoon['pre'].map(lambda x : x[3])

In [18]:
# 언론성향 열 추가
yoon['언론성향'] = yoon['agency'].map(lambda x : classify_agency(x))

In [19]:
yoon.head()

,Unnamed: 0,url,title,agency,content,comment,pre,preprocessed_content_LDA,preprocessed_content_WC,content_noun,content_verb,언론성향
0,0,https://n.news.naver.com/mnews/article/079/000...,"尹, 與 워크숍 3년 연속 참석…""당과 한몸으로 뼈 빠지게 뛰겠다""",노컷뉴스,"尹대통령, 제22대 국민의힘 국회의원 워크숍 참석\n'당정 일체' 강조하며 ""나라를...","['🥵🥸🤯😎어휴 저 병원신 같은놈...귀신은 뭐하냐 저런거 안잡아가고...', '특...","([대, 국민의힘, 국회의원, 워크숍, 참석, 당정, 일체, 강조, 나라, 지키, ...","[대, 국민의힘, 국회의원, 워크숍, 참석, 당정, 일체, 강조, 나라, 지키, 개...",대 국민의힘 국회의원 워크숍 참석 당정 일체 강조 나라 지키 개혁 자 연속 국민의힘...,"[대통령, 국민의힘, 국회의원, 워크숍, 참석, 당정, 강조, 나라, 개혁, 연속,...","[지키, 열리, 하, 있, 내세우, 지키, 밝히, 빠지, 뛰, 힘주, 하, 하, 보...",중도
1,1,https://n.news.naver.com/mnews/article/018/000...,尹대통령 “우주 개척시대 ‘스페이스 스탠다드’ 선도할 것”,이데일리,"우주항공청 개청식 참석…우주기술, 최첨단 기술 집합체\n달·화성 착륙 ‘스페이스 광...","['깝치기보단 내실을. 경제는 왜국에 파냐.?', '우주시대 선도 선언!( R&D ...","([우주, 항공, 청, 개청식, 참석, 우주, 기술, 최, 첨단, 기술, 집합체, ...","[우주, 항공, 청, 개청식, 참석, 우주, 기술, 최, 첨단, 기술, 집합체, 달...",우주 항공 청 개청식 참석 우주 기술 최 첨단 기술 집합체 달 화성 착륙 스페이스 ...,"[우주, 항공, 청, 개청식, 참석, 우주, 기술, 첨단, 기술, 집합체, 달, 화...","[만들, 이어지, 하, 열리, 가, 하, 치, 있, 열리, 향하, 이, 밝히, 가,...",보수
2,2,https://n.news.naver.com/mnews/article/079/000...,"尹, UAE 대통령과 '경제협력' 강화…'새로운 중동붐' 모멘텀",노컷뉴스,"尹대통령, 첫 방한 UAE 대통령과 정상회담\n아랍 국가 최초로 포괄적경제동반자협정...","['이번 계약서 전문 공개해라, 쥐박이 처럼 이면 계약으로 유사시 국군파병 조약 쳐...","([첫, 방한, 정상, 회담, 아랍, 국가, 최초, 포괄, 경제, 동반자, 협정, ...","[첫, 방한, 정상, 회담, 아랍, 국가, 최초, 포괄, 경제, 동반자, 협정, 체...",첫 방한 정상 회담 아랍 국가 최초 포괄 경제 동반자 협정 체결 방문 약속 받 내 ...,"[대통령, 방한, 대통령, 정상, 회담, 아랍, 국가, 최초, 포괄, 경제, 동반자...","[내, 일, 열리, 있, 꼽히, 밝히, 가지, 이, 걸치, 되, 보, 되, 두, 있...",중도
3,3,https://n.news.naver.com/mnews/article/022/000...,장관 사라진 여가부 여성정책도 사라졌다 [심층기획-여가부 장관 공백 사태 100일],세계일보,수장 없는 부처 ‘있으나 마나’… 표류하는 성평등·여성정책\n\n차관 직무대행 체제...,['여가부 20여년의 성과는 지금 출산율과 젠더 갈등으로 증명되지 않았나? 없어지는...,"([수장, 부처, 으나, 표류, 성, 평등, 여성, 정책, 차관, 직무, 대행, 체...","[수장, 부처, 으나, 표류, 성, 평등, 여성, 정책, 차관, 직무, 대행, 체제...",수장 부처 으나 표류 성 평등 여성 정책 차관 직무 대행 체제 운영 현상 유지 교제...,"[수장, 부처, 표류, 성, 평등, 여성, 정책, 차관, 직무, 대행, 체제, 운영...","[없, 있, 말, 하, 빠지, 열, 없, 없, 이어지, 있, 지, 않, 있, 하, ...",알 수 없음
4,4,https://n.news.naver.com/mnews/article/079/000...,"尹대통령 부부, UAE 대통령과 친교 일정…창덕궁 산책 후 만찬",노컷뉴스,UAE 대통령 최초 국빈방문\n윤석열 대통령이 28일 서울 종로구 창덕궁 후원 부용...,"['가가멜과 스머프가 떠오르는 건 뭐지....', '정치 경재는 알고 국빈들하고 대...","([최초, 국빈, 방문, 서울, 종로구, 창덕궁, 후원, 부용지, 일원, 무함마드,...","[최초, 국빈, 방문, 서울, 종로구, 창덕궁, 후원, 부용지, 일원, 무함마드, ...",최초 국빈 방문 서울 종로구 창덕궁 후원 부용지 일원 무함마드 빈 자예드 나흐얀 아...,"[대통령, 최초, 국빈, 방문, 윤석열, 대통령, 서울, 종로구, 창덕궁, 후원, ...","[일, 하, 있, 이, 다지, 전하, 나누, 담, 이, 전하, 내려오, 추, 이, ...",중도


In [21]:
# 최종 전처리본 저장
yoon.to_csv('news_data_윤_P.csv', encoding = 'utf-8')

### 문재인 데이터 전처리

In [22]:
# 대통령별 stopword
stopwords3 = ["대통령", "전대통령","주년","정부","무단","전재","배포", "문재인", "문 대통령", "문"]

In [23]:
# 크롤링 데이터 불러오기
moon = pd.read_csv('news_data_문_final.csv', encoding = 'utf-8')

# pre열 만들어 LDA용, 워드클라우드용, 명사, 동사 순으로 저장
moon['pre'] =  moon['content'].map(lambda x : preprocess_hangul(x))

# LDA용 전처리 열 추가
moon['preprocessed_content_LDA'] =  moon['pre'].map(lambda x : x[0])

# 워드 클라우드용 전처리 열 추가
moon['preprocessed_content_WC'] = moon['pre'].map(lambda x : x[1])

# 언론성향 열 추가
moon['언론성향'] = moon['agency'].map(lambda x : classify_agency(x))

# 명사만 남기기
moon['content_noun'] = moon['pre'].map(lambda x : x[2])

# 용언만 남기기
moon['content_verb'] = moon['pre'].map(lambda x : x[3])

#데이터 프레임 확인
moon.head()

,Unnamed: 0.1,Unnamed: 0,url,title,agency,content,comment,pre,preprocessed_content_LDA,preprocessed_content_WC,언론성향,content_noun,content_verb
0,0,0,https://n.news.naver.com/mnews/article/081/000...,"“文 대통령, 일본에 따박따박 대응하라고 했다” 청와대 비서관 회고",서울신문,문재인 전 대통령이 19일 오후 서울 영등포구 63빌딩에서 열린 9·19 평양공동선...,['중국몽 정신병자.. 왜 북한이나 중국에는 따박따박 대응 못하고 자국 기자가 공안...,"([전, 오후, 서울, 영등포구, 빌딩, 열리, 평양, 공동, 선언, 기념식, 참석...","[전, 오후, 서울, 영등포구, 빌딩, 열리, 평양, 공동, 선언, 기념식, 참석,...",전 오후 서울 영등포구 빌딩 열리 평양 공동 선언 기념식 참석 인사말 사진 공동 취...,알 수 없음,"[문재인, 대통령, 일, 오후, 서울, 영등포구, 빌딩, 평양, 공동, 선언, 기념...","[열리, 하, 있, 펴내, 담, 담기, 있, 지나, 맡, 달, 하, 하, 맡, 이,..."
1,1,1,https://n.news.naver.com/mnews/article/421/000...,"文·尹·李 이어 한동훈도 ""5·18정신 헌법수록""…그런데 왜 아직까지?",뉴스1,"헌법 자문위 ""합당치 않다"" 의견… 文대통령 개헌안 '자동폐기'\n5·18관계자 ""...","['윤석열/한동훈 전두환/노태우 를 보는듯', '헌법은 커녕 망언이나 안하면 그나마...","([헌법, 자문위, 합당, 의견, 개헌안, 자동, 폐기, 관계자, 정치, 이용, 그...","[헌법, 자문위, 합당, 의견, 개헌안, 자동, 폐기, 관계자, 정치, 이용, 그만...",헌법 자문위 합당 의견 개헌안 자동 폐기 관계자 정치 이용 그만 적극 속도감 움직임...,알 수 없음,"[헌법, 자문위, 의견, 대통령, 개헌안, 자동, 폐기, 관계자, 정치, 이용, 적...","[않, 있, 서, 있, 대하, 이, 밝히, 이, 만나, 지키, 이, 들어가, 지, ..."
2,2,2,https://n.news.naver.com/mnews/article/366/000...,‘김신조 사건’ 54년 만에 靑 뒤 북악산 전면 개방…文대통령 부부 산행,조선비즈,"靑 “文대통령, 대선 때 북악산 전면 개방 공약”\n“국민과의 약속 지킨 것”…20...","['여테있다가 새정부 개방한다닌깐ᆢ좀스럽다ᆢ좌파들', '니가 개방하냐, 숟가락 얻는...","([대선, 북악산, 전면, 개방, 공약, 국민, 약속, 지키, 순차, 개방, 여사,...","[대선, 북악산, 전면, 개방, 공약, 국민, 약속, 지키, 순차, 개방, 여사, ...",대선 북악산 전면 개방 공약 국민 약속 지키 순차 개방 여사 부부 산행 청와대 건물...,보수,"[대통령, 대선, 때, 북악산, 전면, 개방, 공약, 국민, 약속, 년, 순차, 개...","[지키, 오, 있, 되, 통하, 나누, 있, 앞두, 지나, 대하, 있, 남, 있, ..."
3,3,3,https://n.news.naver.com/mnews/article/003/001...,집무실 아래까지 마중 나온 文대통령…尹당선인 '극진 예우'(종합),뉴시스,기사내용 요약\n1년 9개월만에 대면한 文…엷은 미소로 尹에 악수\n상춘재까지 걸으...,['사치킹 필리핀 이멜다 징역 77년 선고ㅋㅋㅋㅋ K-이멜다 김돼숙도 징역 77년 ...,"([기사, 내용, 요약, 개월, 대면, 엷, 미소, 악수, 상, 춘제, 걷, 으며,...","[기사, 내용, 요약, 개월, 대면, 엷, 미소, 악수, 상, 춘제, 걷, 으며, ...",기사 내용 요약 개월 대면 엷 미소 악수 상 춘제 걷 으며 경내 설명 정원 너머 헬...,중도,"[기사, 내용, 요약, 대면, 미소, 악수, 상, 춘제, 경내, 설명, 정원, 너머...","[엷, 피, 앞서, 나누, 있, 위하, 나가, 기다리, 하, 있, 여미, 내려오, ..."
4,4,4,https://n.news.naver.com/mnews/article/366/000...,文대통령 “국회의원 적성에 맞나” 고민정 “열심히 싸우고 있다”,조선비즈,文대통령 “상대방에 의해 정의되지 말고 내가 왜 하는지를 찾아야지요”\n\n문재인 ...,"['생각없는 의원이군. 싸우려고 국회의원하나?', '북한 주체사상 영향 받은 애들 ...","([상대방, 의하, 정의, 는지, 찾, 어야지요, 청와대, 대변인, 출신, 고민정,...","[상대방, 의하, 정의, 는지, 찾, 어야지요, 청와대, 대변인, 출신, 고민정, ...",상대방 의하 정의 는지 찾 어야지요 청와대 대변인 출신 고민정 더불어민주당 의원 국...,보수,"[대통령, 상대방, 정의, 문재인, 대통령, 청와대, 대변인, 출신, 고민정, 더불...","[의하, 말, 하, 찾, 맞, 이, 모르, 싸우, 있, 이, 알려지, 있, 올리, ..."


In [24]:
moon.to_csv('news_data_문_P.csv', encoding = 'utf-8')

### 박근혜 데이터 전처리

In [25]:
# 대통령별 stopword
stopwords3 = ["대통령", "전대통령","주년","정부","박","무단","전재","배포", "박근혜", "박 대통령"]

In [26]:
# 크롤링 데이터 불러오기
park = pd.read_csv('news_data_박_final.csv', encoding = 'utf-8')

# pre열 만들어 LDA용, 워드클라우드용, 명사, 동사 순으로 저장
park['pre'] =  park['content'].map(lambda x : preprocess_hangul(x))

# LDA용 전처리 열 추가
park['preprocessed_content_LDA'] =  park['pre'].map(lambda x : x[0])

# 워드 클라우드용 전처리 열 추가
park['preprocessed_content_WC'] = park['pre'].map(lambda x : x[1])

# 언론성향 열 추가
park['언론성향'] = park['agency'].map(lambda x : classify_agency(x))

# 명사만 남기기
park['content_noun'] = park['pre'].map(lambda x : x[2])

# 용언만 남기기
park['content_verb'] = park['pre'].map(lambda x : x[3])

#데이터 프레임 확인
park.head()

,Unnamed: 0.1,Unnamed: 0,url,title,agency,content,comment,pre,preprocessed_content_LDA,preprocessed_content_WC,언론성향,content_noun,content_verb
0,0,0,https://n.news.naver.com/mnews/article/081/000...,"“文 대통령, 일본에 따박따박 대응하라고 했다” 청와대 비서관 회고",서울신문,문재인 전 대통령이 19일 오후 서울 영등포구 63빌딩에서 열린 9·19 평양공동선...,['중국몽 정신병자.. 왜 북한이나 중국에는 따박따박 대응 못하고 자국 기자가 공안...,"([문재인, 전, 오후, 서울, 영등포구, 빌딩, 열리, 평양, 공동, 선언, 기념...","[문재인, 전, 오후, 서울, 영등포구, 빌딩, 열리, 평양, 공동, 선언, 기념식...",문재인 전 오후 서울 영등포구 빌딩 열리 평양 공동 선언 기념식 참석 인사말 사진 ...,알 수 없음,"[문재인, 대통령, 일, 오후, 서울, 영등포구, 빌딩, 평양, 공동, 선언, 기념...","[열리, 하, 있, 펴내, 담, 담기, 있, 지나, 맡, 달, 하, 하, 맡, 이,..."
1,1,1,https://n.news.naver.com/mnews/article/421/000...,"文·尹·李 이어 한동훈도 ""5·18정신 헌법수록""…그런데 왜 아직까지?",뉴스1,"헌법 자문위 ""합당치 않다"" 의견… 文대통령 개헌안 '자동폐기'\n5·18관계자 ""...","['윤석열/한동훈 전두환/노태우 를 보는듯', '헌법은 커녕 망언이나 안하면 그나마...","([헌법, 자문위, 합당, 의견, 개헌안, 자동, 폐기, 관계자, 정치, 이용, 그...","[헌법, 자문위, 합당, 의견, 개헌안, 자동, 폐기, 관계자, 정치, 이용, 그만...",헌법 자문위 합당 의견 개헌안 자동 폐기 관계자 정치 이용 그만 적극 속도감 움직임...,알 수 없음,"[헌법, 자문위, 의견, 대통령, 개헌안, 자동, 폐기, 관계자, 정치, 이용, 적...","[않, 있, 서, 있, 대하, 이, 밝히, 이, 만나, 지키, 이, 들어가, 지, ..."
2,2,2,https://n.news.naver.com/mnews/article/366/000...,‘김신조 사건’ 54년 만에 靑 뒤 북악산 전면 개방…文대통령 부부 산행,조선비즈,"靑 “文대통령, 대선 때 북악산 전면 개방 공약”\n“국민과의 약속 지킨 것”…20...","['여테있다가 새정부 개방한다닌깐ᆢ좀스럽다ᆢ좌파들', '니가 개방하냐, 숟가락 얻는...","([대선, 북악산, 전면, 개방, 공약, 국민, 약속, 지키, 순차, 개방, 여사,...","[대선, 북악산, 전면, 개방, 공약, 국민, 약속, 지키, 순차, 개방, 여사, ...",대선 북악산 전면 개방 공약 국민 약속 지키 순차 개방 여사 부부 산행 청와대 건물...,보수,"[대통령, 대선, 때, 북악산, 전면, 개방, 공약, 국민, 약속, 년, 순차, 개...","[지키, 오, 있, 되, 통하, 나누, 있, 앞두, 지나, 대하, 있, 남, 있, ..."
3,3,3,https://n.news.naver.com/mnews/article/003/001...,집무실 아래까지 마중 나온 文대통령…尹당선인 '극진 예우'(종합),뉴시스,기사내용 요약\n1년 9개월만에 대면한 文…엷은 미소로 尹에 악수\n상춘재까지 걸으...,['사치킹 필리핀 이멜다 징역 77년 선고ㅋㅋㅋㅋ K-이멜다 김돼숙도 징역 77년 ...,"([기사, 내용, 요약, 개월, 대면, 엷, 미소, 악수, 상, 춘제, 걷, 으며,...","[기사, 내용, 요약, 개월, 대면, 엷, 미소, 악수, 상, 춘제, 걷, 으며, ...",기사 내용 요약 개월 대면 엷 미소 악수 상 춘제 걷 으며 경내 설명 정원 너머 헬...,중도,"[기사, 내용, 요약, 대면, 미소, 악수, 상, 춘제, 경내, 설명, 정원, 너머...","[엷, 피, 앞서, 나누, 있, 위하, 나가, 기다리, 하, 있, 여미, 내려오, ..."
4,4,4,https://n.news.naver.com/mnews/article/366/000...,文대통령 “국회의원 적성에 맞나” 고민정 “열심히 싸우고 있다”,조선비즈,文대통령 “상대방에 의해 정의되지 말고 내가 왜 하는지를 찾아야지요”\n\n문재인 ...,"['생각없는 의원이군. 싸우려고 국회의원하나?', '북한 주체사상 영향 받은 애들 ...","([상대방, 의하, 정의, 는지, 찾, 어야지요, 문재인, 청와대, 대변인, 출신,...","[상대방, 의하, 정의, 는지, 찾, 어야지요, 문재인, 청와대, 대변인, 출신, ...",상대방 의하 정의 는지 찾 어야지요 문재인 청와대 대변인 출신 고민정 더불어민주당 ...,보수,"[대통령, 상대방, 정의, 문재인, 대통령, 청와대, 대변인, 출신, 고민정, 더불...","[의하, 말, 하, 찾, 맞, 이, 모르, 싸우, 있, 이, 알려지, 있, 올리, ..."


In [27]:
park.to_csv('news_data_박_P.csv', encoding = 'utf-8')

### 이명박 데이터 전처리

In [29]:
# 대통령별 stopword
stopwords3 = ["대통령", "전대통령","주년","정부","이","무단","전재","배포", "이명박", "이 대통령"]

In [30]:
# 크롤링 데이터 불러오기
lee = pd.read_csv('news_data_이_final.csv', encoding = 'utf-8')

# pre열 만들어 LDA용, 워드클라우드용, 명사, 동사 순으로 저장
lee['pre'] =  lee['content'].map(lambda x : preprocess_hangul(x))

# LDA용 전처리 열 추가
lee['preprocessed_content_LDA'] =  lee['pre'].map(lambda x : x[0])

# 워드 클라우드용 전처리 열 추가
lee['preprocessed_content_WC'] = lee['pre'].map(lambda x : x[1])

# 언론성향 열 추가
lee['언론성향'] = lee['agency'].map(lambda x : classify_agency(x))

# 명사만 남기기
lee['content_noun'] = lee['pre'].map(lambda x : x[2])

# 용언만 남기기
lee['content_verb'] = lee['pre'].map(lambda x : x[3])

#데이터 프레임 확인
lee.head()

,Unnamed: 0,url,title,agency,content,comment,pre,preprocessed_content_LDA,preprocessed_content_WC,언론성향,content_noun,content_verb
0,0,https://n.news.naver.com/mnews/article/029/000...,`3전4기` 끝 성공한 의대 증원…이명박·문재인 정부 번번이 실패,디지털타임스,"의대 증원, 1998년이 마지막\n\n의약분업 때 줄어든 정원, 19년간 '동결'\...",['밥그릇 챙기려고 환자 목숨 볼모로 정부와 국민 협박하는 의사 집단에는 여야 가리...,"([의대, 증원, 마지막, 의약, 분업, 줄, 정원, 간, 동결, 의사, 반발, 무...","[의대, 증원, 마지막, 의약, 분업, 줄, 정원, 간, 동결, 의사, 반발, 무릅...",의대 증원 마지막 의약 분업 줄 정원 간 동결 의사 반발 무릅쓰 의대 증원 확정 다...,보수,"[의대, 증원, 마지막, 의약, 분업, 때, 정원, 간, 동결, 의사, 반발, 의대...","[줄, 들, 무릅쓰, 앞두, 가, 늘어나, 인하, 강하, 물러서, 하, 들, 거두,..."
1,1,https://n.news.naver.com/mnews/article/003/001...,"여 지도부, 이명박 예방…이 ""어려운 시기에 잘 단합해야""(종합)",뉴시스,"황우여·추경호, 연달아 MB 사저 방문해 면담\nMB ""정부와 사전 조율·일치된 모...",[],"([황우여, 추경호, 연달, 사저, 방문, 면담, 사전, 조율, 일치, 모습, 보이...","[황우여, 추경호, 연달, 사저, 방문, 면담, 사전, 조율, 일치, 모습, 보이,...",황우여 추경호 연달 사저 방문 면담 사전 조율 일치 모습 보이 서울 뉴시스 권창회 ...,중도,"[황우여, 추경호, 사저, 방문, 면담, 정부, 사전, 조율, 일치, 모습, 서울,...","[연달, 보이, 비롯하, 찾, 하, 있, 하, 전하, 하, 있, 찾, 마치, 나오,..."
2,2,https://n.news.naver.com/mnews/article/421/000...,"""노무현·이명박도 특검 받아들였다""…민주, 與 이탈표 챙기기",뉴스1,"尹'10번째 거부권' 행사…野 단독처리·특검추천 등 4가지 이유\n민주 ""과거 특검...",['정부가 잘못한건 제대로 수사하고 고치도록해야하는데 숨기기만 급급하니 어느새 특검...,"([번째, 거부, 권, 행사, 단독, 처리, 특검, 추천, 가지, 이유, 민주, 과...","[번째, 거부, 권, 행사, 단독, 처리, 특검, 추천, 가지, 이유, 민주, 과거...",번째 거부 권 행사 단독 처리 특검 추천 가지 이유 민주 과거 특검 전례 존재 본회...,알 수 없음,"[거부, 행사, 단독, 처리, 특검, 추천, 이유, 민주, 과거, 특검, 전례, 존...","[재, 일, 열리, 하, 있, 이, 위하, 두, 가지, 있, 내세우, 이, 따지, ..."
3,3,https://n.news.naver.com/mnews/article/014/000...,"국힘 황우여·추경호, 내일 이명박 전 대통령 예방",파이낸셜뉴스,비대위 회의 입장하는 황우여 비대위원장과 추경호 원내대표 (서울=연합뉴스) 김주형 ...,[],"([비대위, 회의, 입장, 황우여, 비대위원장, 추경호, 원내, 대표, 서울, 연합...","[비대위, 회의, 입장, 황우여, 비대위원장, 추경호, 원내, 대표, 서울, 연합뉴...",비대위 회의 입장 황우여 비대위원장 추경호 원내 대표 서울 연합뉴스 김주형 기자 국...,보수,"[비대위, 회의, 입장, 황우여, 비대위원장, 추경호, 원내, 대표, 서울, 연합뉴...","[열리, 있, 오, 지나, 이뤄지, 이, 오, 찾, 가, 이, 알려지, 앞서, 이]"
4,4,https://n.news.naver.com/mnews/article/052/000...,"황우여 만난 이명박 ""일치된 여당다운 모습 보여야""",YTN,국민의힘 황우여 비상대책위원장을 만난 이명박 전 대통령은 정부와 사전조율을 통해 일...,[],"([국민의힘, 황우여, 비상, 대책, 위원장, 만나, 전, 사전, 조율, 일치, 여...","[국민의힘, 황우여, 비상, 대책, 위원장, 만나, 전, 사전, 조율, 일치, 여당...",국민의힘 황우여 비상 대책 위원장 만나 전 사전 조율 일치 여당 답 모습 보이 달 ...,중도,"[국민의힘, 황우여, 비상, 대책, 위원장, 이명박, 대통령, 정부, 사전, 조율,...","[만나, 통하, 보이, 달, 찾, 만나, 이, 합치, 달, 있, 밝히, 모시, 싶,..."


In [31]:
lee.to_csv('news_data_이_P.csv', encoding = 'utf-8')

### 노무현 데이터 전처리

In [32]:
# 대통령별 stopword
stopwords3 = ["대통령", "전대통령", "주년", "정부", "노", "무단", "전재", "배포", "노무현", "노 대통령"]

In [33]:
# 크롤링 데이터 불러오기
noh = pd.read_csv('news_data_노_final.csv', encoding = 'utf-8')

# pre열 만들어 LDA용, 워드클라우드용, 명사, 동사 순으로 저장
noh['pre'] =  noh['content'].map(lambda x : preprocess_hangul(x))

# LDA용 전처리 열 추가
noh['preprocessed_content_LDA'] =  noh['pre'].map(lambda x : x[0])

# 워드 클라우드용 전처리 열 추가
noh['preprocessed_content_WC'] = noh['pre'].map(lambda x : x[1])

# 언론성향 열 추가
noh['언론성향'] = noh['agency'].map(lambda x : classify_agency(x))

# 명사만 남기기
noh['content_noun'] = noh['pre'].map(lambda x : x[2])

# 용언만 남기기
noh['content_verb'] = noh['pre'].map(lambda x : x[3])

#데이터 프레임 확인
noh.head()

,Unnamed: 0,url,title,content,comment,agency,pre,preprocessed_content_LDA,preprocessed_content_WC,언론성향,content_noun,content_verb
0,0,https://n.news.naver.com/mnews/article/658/000...,[세상읽기] 노무현이라면…,박재율 지방분권균형발전 부산시민연대 상임대표·지방분권전국회의 공동대표\n노무현.\n...,"['국민은 정치를 생각도안하는 그런나라를 꿈꾸시겠지.♡', '노무현은 강성노조 지원...",국제신문,"([박제, 율, 지방, 분권, 균형, 발전, 부산, 시민, 연대, 상임, 대표, 지...","[박제, 율, 지방, 분권, 균형, 발전, 부산, 시민, 연대, 상임, 대표, 지방...",박제 율 지방 분권 균형 발전 부산 시민 연대 상임 대표 지방 분권 전국 회의 공동...,보수,"[박제, 지방, 분권, 균형, 발전, 부산, 시민, 연대, 상임, 대표, 지방, 분...","[떠오르, 이, 아니, 지나, 거치, 보, 위하, 않, 하, 우직하, 다가오, 헤치..."
1,1,https://n.news.naver.com/mnews/article/056/001...,"이재명, 노무현 서거 15주기 맞아 “당원 중심 정당, 반드시 가야 할 미래”",더불어민주당 이재명 대표는 고(故) 노무현 대통령 서거 15주기를 맞아 “깨어 있는...,['나라를 지키다 순국한 장병들 추모는 안하는것들이 죄짓고 혼자가신 분은 매년 추모...,KBS,"([더불어민주당, 이재명, 대표, 고, 서거, 주기, 맞, 깨, 시민, 함께, 만들...","[더불어민주당, 이재명, 대표, 고, 서거, 주기, 맞, 깨, 시민, 함께, 만들,...",더불어민주당 이재명 대표 고 서거 주기 맞 깨 시민 함께 만들 내 참여 정치 시대 ...,중도,"[더불어민주당, 이재명, 대표, 노무현, 대통령, 서거, 주기, 시민, 참여, 정치...","[맞, 깨, 있, 만들, 내, 않, 모르, 나아가, 하, 이, 밝히, 깨, 있, 앞..."
2,2,https://n.news.naver.com/mnews/article/629/000...,노무현 대통령 서거 15주기 특별사진전 개막,"노무현재단 광주지역위원회 주관, 국립아시아문화전당 플라자브릿지서 열려\n\n20일 ...",['시대가 암울하니 더욱 그리워 집니다. 내 마음속 대통령 노무현.'],더팩트,"([노무현재단, 광주, 위원회, 주관, 국립아시아문화전당, 플라자, 브릿지, 서, ...","[노무현재단, 광주, 위원회, 주관, 국립아시아문화전당, 플라자, 브릿지, 서, 열...",노무현재단 광주 위원회 주관 국립아시아문화전당 플라자 브릿지 서 열리 재단 광주 위...,알 수 없음,"[노무현재단, 광주, 지역, 위원회, 주관, 국립아시아문화전당, 플라자, 브릿지, ...","[열리, 열리, 열리, 오, 서, 오, 뛰, 기다리]"
3,3,https://n.news.naver.com/mnews/article/028/000...,"박정희 향수가 낳은 박근혜 팬덤, 노무현 애수가 낳은 문재인 팬덤",이철희의 돌아보고 내다보고 06 _박팬덤과 문팬덤\n2017년 3월4일 박근혜 대통...,"['간만에 냉철한,상식적인 글 읽고갑니다. 우리사회가 이런 스탠스를 가질때 건강해진...",한겨레,"([이철희, 돌아보, 내다보, 박, 팬덤, 문, 팬덤, 박근혜, 탄핵, 기각, 요구...","[이철희, 돌아보, 내다보, 박, 팬덤, 문, 팬덤, 박근혜, 탄핵, 기각, 요구,...",이철희 돌아보 내다보 박 팬덤 문 팬덤 박근혜 탄핵 기각 요구 태극기 집회 박정희 ...,진보,"[이철희, 박, 팬덤, 문, 팬덤, 월, 일, 박근혜, 대통령, 탄핵, 기각, 요구...","[돌아보, 내다보, 열리, 있, 같, 열리, 있, 대하, 있, 하, 있, 하, 먹,..."
4,4,https://n.news.naver.com/mnews/article/022/000...,"""부자 감세""라더니…민주당, 종부세 손질 논의 예고",이재명式 실용정치 신호탄 되나\n\n박찬대 이어 고민정 개편론 제기\n비공개 최고위...,"['금투세 폐지 부탁드립니다~~~', '금투세는 폐지하자', '부자 감세라고 종부세...",세계일보,"([이재명, 실용, 정치, 신호탄, 박차, 잇, 고민정, 개편, 론, 제기, 비, ...","[이재명, 실용, 정치, 신호탄, 박차, 잇, 고민정, 개편, 론, 제기, 비, 공...",이재명 실용 정치 신호탄 박차 잇 고민정 개편 론 제기 비 공개 최고 위 서 관련 ...,알 수 없음,"[이재명, 실용, 정치, 신호탄, 고민정, 개편, 제기, 공개, 최고, 위, 서, ...","[되, 박차, 나누, 대하, 잇따르, 내놓, 있, 대하, 이, 오, 이, 둘러싸, ..."


In [34]:
noh.to_csv('news_data_노_P.csv', encoding = 'utf-8')

### 데이터 개수 확인

In [36]:
yoon.count()

Unnamed: 0                  3575
url                         3575
title                       3575
agency                      3575
content                     3575
comment                     3575
pre                         3575
preprocessed_content_LDA    3575
preprocessed_content_WC     3575
content_noun                3575
content_verb                3575
언론성향                        3575
dtype: int64

In [37]:
moon.count()

Unnamed: 0.1                3406
Unnamed: 0                  3406
url                         3406
title                       3406
agency                      3406
content                     3406
comment                     3406
pre                         3406
preprocessed_content_LDA    3406
preprocessed_content_WC     3406
언론성향                        3406
content_noun                3406
content_verb                3406
dtype: int64

In [40]:
park.count()

Unnamed: 0.1    2970
Unnamed: 0      2970
url             2970
title           2970
agency          2970
content         2970
comment         2970
dtype: int64

In [41]:
lee.count()

Unnamed: 0                  4828
url                         4828
title                       4828
agency                      4828
content                     4828
comment                     4828
pre                         4828
preprocessed_content_LDA    4828
preprocessed_content_WC     4828
언론성향                        4828
content_noun                4828
content_verb                4828
dtype: int64

In [42]:
noh.count()

Unnamed: 0                  5031
url                         5031
title                       5031
content                     5031
comment                     5031
agency                      5031
pre                         5031
preprocessed_content_LDA    5031
preprocessed_content_WC     5031
언론성향                        5031
content_noun                5031
content_verb                5031
dtype: int64

### 총 19180개 데이터 수집 완료